# Chapter 7 - Ensemble Learning and Random Forests


## Pre-excercise

In [7]:
import pandas as pd

data_import = pd.read_csv("Placement_Data_Full_Class.csv")
data_import

,sl_no,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,salary
0,1,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed,270000.0
1,2,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed,200000.0
2,3,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed,250000.0
3,4,M,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed,NaN
4,5,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed,425000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,211,M,80.60,Others,82.00,Others,Commerce,77.60,Comm&Mgmt,No,91.0,Mkt&Fin,74.49,Placed,400000.0
211,212,M,58.00,Others,60.00,Others,Science,72.00,Sci&Tech,No,74.0,Mkt&Fin,53.62,Placed,275000.0
212,213,M,67.00,Others,67.00,Others,Commerce,73.00,Comm&Mgmt,Yes,59.0,Mkt&Fin,69.72,Placed,295000.0
213,214,F,74.00,Others,66.00,Others,Commerce,58.00,Comm&Mgmt,No,70.0,Mkt&HR,60.23,Placed,204000.0


In [8]:
data_import.describe()

,sl_no,ssc_p,hsc_p,degree_p,etest_p,mba_p,salary
count,215.000000,215.000000,215.000000,215.000000,215.000000,215.000000,148.000000
mean,108.000000,67.303395,66.333163,66.370186,72.100558,62.278186,288655.405405
std,62.209324,10.827205,10.897509,7.358743,13.275956,5.833385,93457.452420
min,1.000000,40.890000,37.000000,50.000000,50.000000,51.210000,200000.000000
25%,54.500000,60.600000,60.900000,61.000000,60.000000,57.945000,240000.000000
50%,108.000000,67.000000,65.000000,66.000000,71.000000,62.000000,265000.000000
75%,161.500000,75.700000,73.000000,72.000000,83.500000,66.255000,300000.000000
max,215.000000,89.400000,97.700000,91.000000,98.000000,77.890000,940000.000000


In [9]:
for column in data_import.columns:
    print(column, data_import[column].dtype, len(data_import[column].unique()))

sl_no int64 215
gender object 2
ssc_p float64 103
ssc_b object 2
hsc_p float64 97
hsc_b object 2
hsc_s object 3
degree_p float64 89
degree_t object 3
workex object 2
etest_p float64 100
specialisation object 2
mba_p float64 205
status object 2
salary float64 46


In [10]:
dropped_columns = data_import.copy()

dropped_columns = dropped_columns.drop(["sl_no", "salary"], axis=1)
dropped_columns

,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status
0,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed
1,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed
2,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed
3,M,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed
4,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed
...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,M,80.60,Others,82.00,Others,Commerce,77.60,Comm&Mgmt,No,91.0,Mkt&Fin,74.49,Placed
211,M,58.00,Others,60.00,Others,Science,72.00,Sci&Tech,No,74.0,Mkt&Fin,53.62,Placed
212,M,67.00,Others,67.00,Others,Commerce,73.00,Comm&Mgmt,Yes,59.0,Mkt&Fin,69.72,Placed
213,F,74.00,Others,66.00,Others,Commerce,58.00,Comm&Mgmt,No,70.0,Mkt&HR,60.23,Placed


In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label_encoded_data = dropped_columns.copy()
columns_to_label_encode = ["gender", "workex", "ssc_b", "hsc_b", "specialisation"]
label_encoded_data[columns_to_label_encode] = label_encoded_data[columns_to_label_encode].apply(le.fit_transform)
label_encoded_data

,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status
0,1,67.00,1,91.00,1,Commerce,58.00,Sci&Tech,0,55.0,1,58.80,Placed
1,1,79.33,0,78.33,1,Science,77.48,Sci&Tech,1,86.5,0,66.28,Placed
2,1,65.00,0,68.00,0,Arts,64.00,Comm&Mgmt,0,75.0,0,57.80,Placed
3,1,56.00,0,52.00,0,Science,52.00,Sci&Tech,0,66.0,1,59.43,Not Placed
4,1,85.80,0,73.60,0,Commerce,73.30,Comm&Mgmt,0,96.8,0,55.50,Placed
...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,1,80.60,1,82.00,1,Commerce,77.60,Comm&Mgmt,0,91.0,0,74.49,Placed
211,1,58.00,1,60.00,1,Science,72.00,Sci&Tech,0,74.0,0,53.62,Placed
212,1,67.00,1,67.00,1,Commerce,73.00,Comm&Mgmt,1,59.0,0,69.72,Placed
213,0,74.00,1,66.00,1,Commerce,58.00,Comm&Mgmt,0,70.0,1,60.23,Placed


In [13]:
hot_encoded_data = label_encoded_data.copy()

hot_encoded_data_y_placeholder = hot_encoded_data["status"]
hot_encoded_data = hot_encoded_data.drop("status", axis=1)

hot_encoded_data = pd.get_dummies(hot_encoded_data)

hot_encoded_data = pd.concat([hot_encoded_data, hot_encoded_data_y_placeholder], axis=1)
hot_encoded_data

,gender,ssc_p,ssc_b,hsc_p,hsc_b,degree_p,workex,etest_p,specialisation,mba_p,hsc_s_Arts,hsc_s_Commerce,hsc_s_Science,degree_t_Comm&Mgmt,degree_t_Others,degree_t_Sci&Tech,status
0,1,67.00,1,91.00,1,58.00,0,55.0,1,58.80,0,1,0,0,0,1,Placed
1,1,79.33,0,78.33,1,77.48,1,86.5,0,66.28,0,0,1,0,0,1,Placed
2,1,65.00,0,68.00,0,64.00,0,75.0,0,57.80,1,0,0,1,0,0,Placed
3,1,56.00,0,52.00,0,52.00,0,66.0,1,59.43,0,0,1,0,0,1,Not Placed
4,1,85.80,0,73.60,0,73.30,0,96.8,0,55.50,0,1,0,1,0,0,Placed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,1,80.60,1,82.00,1,77.60,0,91.0,0,74.49,0,1,0,1,0,0,Placed
211,1,58.00,1,60.00,1,72.00,0,74.0,0,53.62,0,0,1,0,0,1,Placed
212,1,67.00,1,67.00,1,73.00,1,59.0,0,69.72,0,1,0,1,0,0,Placed
213,0,74.00,1,66.00,1,58.00,0,70.0,1,60.23,0,1,0,1,0,0,Placed


In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = hot_encoded_data.copy()
scaled_data_temp = scaled_data.iloc[:, -1]
scaled_data = scaled_data.iloc[:, :-1]

scaled_data = scaler.fit_transform(scaled_data)

scaled_data

array([[ 0.73943397, -0.02808697,  1.08245885, ..., -1.43924583,
        -0.23221018,  1.62605898],
       [ 0.73943397,  1.11336869, -0.92382264, ..., -1.43924583,
        -0.23221018,  1.62605898],
       [ 0.73943397, -0.21323793, -0.92382264, ...,  0.69480833,
        -0.23221018, -0.61498384],
       ...,
       [ 0.73943397, -0.02808697,  1.08245885, ...,  0.69480833,
        -0.23221018, -0.61498384],
       [-1.35238581,  0.61994138,  1.08245885, ...,  0.69480833,
        -0.23221018, -0.61498384],
       [ 0.73943397, -0.49096436, -0.92382264, ...,  0.69480833,
        -0.23221018, -0.61498384]])

In [16]:
from sklearn.model_selection import train_test_split

X = scaled_data
y = scaled_data_temp

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
X_train

array([[-1.35238581, -0.02808697, -0.92382264, ..., -1.43924583,
        -0.23221018,  1.62605898],
       [ 0.73943397,  1.2605637 ,  1.08245885, ...,  0.69480833,
        -0.23221018, -0.61498384],
       [ 0.73943397, -1.41671915,  1.08245885, ...,  0.69480833,
        -0.23221018, -0.61498384],
       ...,
       [ 0.73943397, -1.69444559,  1.08245885, ..., -1.43924583,
        -0.23221018,  1.62605898],
       [-1.35238581,  0.5273659 , -0.92382264, ...,  0.69480833,
        -0.23221018, -0.61498384],
       [-1.35238581,  1.63827165,  1.08245885, ...,  0.69480833,
        -0.23221018, -0.61498384]])

In [20]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_for_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(estimators = [('lr', log_clf), ('rf', rnd_for_clf), ('sc', svm_clf)], voting="hard")

In [21]:
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('sc', SVC())])

In [22]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_for_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.8591549295774648
RandomForestClassifier 0.9295774647887324
SVC 0.8169014084507042
VotingClassifier 0.8591549295774648


In [23]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1) # Bootstrap determines whether this is bagging or pasting
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

In [24]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rf_clf.fit(X_train, y_train)

for name, score in zip(hot_encoded_data.iloc[:, :-1].columns, rf_clf.feature_importances_):
    print(name, ": ", score)

gender :  0.01913603756449764
ssc_p :  0.249996347762715
ssc_b :  0.019189059973106055
hsc_p :  0.17269814397066122
hsc_b :  0.013114962022634976
degree_p :  0.1739237922902334
workex :  0.05675807586475929
etest_p :  0.08287833835142788
specialisation :  0.029894098401491733
mba_p :  0.12239813072498242
hsc_s_Arts :  0.003925905869942791
hsc_s_Commerce :  0.00977913207699108
hsc_s_Science :  0.012944135326922502
degree_t_Comm&Mgmt :  0.012083338388365913
degree_t_Others :  0.006536676208487919
degree_t_Sci&Tech :  0.014743825202780312


In [25]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200, 
    algorithm="SAMME.R", learning_rate=0.5)

ada_clf.fit(X_train, y_train)

y_pred = ada_clf.predict(X_test)

accuracy_score(y_test, y_pred)

0.8873239436619719

In [26]:
from sklearn.ensemble import GradientBoostingClassifier

gbrt = GradientBoostingClassifier(max_depth=2, n_estimators=120)

gbrt.fit(X_train, y_train)

y_pred = gbrt.predict(X_test)

accuracy_score(y_test, y_pred)

0.9014084507042254

## Main Text
Ensemble: a group of predictors

## Voting Classifiers
*Hard voting* classifier: Aggregating predictions of each classifier and predict the class that gets the most votes. <br>
Even if each classifier is a *weak learner*, the ensemble can still get a *strong learner*, provided there are a sufficient number of weak learners and they are sufficiently diverse due to the *law of large numbers* <br>
This is only true if all classifiers are perfectly independent which is clearly not the case since they are trained on the same data. <br>
*Soft voting*: Predicting the class with the hightest class probability, averaged over all the individual classifiers. <br>
Soft voting often achieves higher performance than hard voting because it gives more weight to highly confident votes.

## Bagging and Pasting
To get a diverse set of classifiers, you can use the same training algorithm for every predictor, but to train them on different random subsets of the training set. <br>
Bagging: sampling is peformed with replacement. <br>
Pasting: sampling is performed without replacement. <br>
The ensemble has a similiar bias but a lower variance than a single predictor trained on the original training set.

### Bagging and and Pasting in Scikit-Learn
An ensemble's predictions will likely generalize much better than a sigle Decision Tree's predictions: the ensemble has a comparable bias but a smaller variance (it makes roughly the same number of errors on the training set, but the decision boundary is less irregular). <br>
Bagging often results in better models because predictors end up being less correlated and the ensemble's variance is reduced.


In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

### Out-of-Bag Evaluation
Out-of-bag (oob) instances: remaining training instances that are not sampled due to replacement in bagging. <br>
Since a predictor never sees the oob instances during training, it can be evaluated on these instances, without the need for a separate validation set. <br>
The oob decision function returns the class probabilties for each training instances.

In [ ]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score=True)

bag_clf.fit(X_train, y_train)

bag_clf.oob_score_

## Random Patches and Random Subspaces
The `BaggingClassifier` class supports sampling the features as well. This is controlled by two hyperparameters: `max_features` and `bootstrap_features`. <br>
The Random Paches method: sampling both training instances and features. <br>
The Random subspaces method: keeping all training instances but sampling features. <br>
Sampling features results in even more predictor diversity, trading a bit more bias for a lower variance.


## Random Forests
A Random Forest is an ensemble of Decision Trees, generally trained via the bagging method (or somethimes pasting), typically with `max_samples` set to the size of the training set. <br>
Instead of building a `BaggingClassifier` and passing it a `DecisionTreeClassifier`, you can instead use the `RandomForestClassifier` class, which is more convenient and optimized for Decision Trees.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

### Extra-Trees
Extremely Randomized Trees (Extra-Trees): making trees random not only by considering a random subset of the features for splitting, but also by using random thresholds for each feature. <br>
This trades more bias for a lower variance. It also makes Extra-Trees much faster to train than regular Random Forests. <br>

### Feature Importance
Scikit-Learn measures a feature's importance by looking at how much the tree nodes that use that feature reduce impurity on average (across all trees in the forest). <br>
Scikit-Learn computes this score automatically for each feature after training, then it scales the results so that the sum of all importances is equal to 1. You can access the result using the `feature_importances_` variable. <br>
Random Forests are very handy to get a quick understanding of what feature actually matter, in particular if you need to perform feature selection.



## Boosting
*Boosting* refers to any Ensemble method that can combine several weak learners into a strong learner. The general idea of most boosting methods is to train predictors sequentially, each trying to correct its predecessor.

### AdaBoost
An AdaBoost classifier gradually improve a predictor by increasing the relative weight of misclassified training instances.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5)
ada_clf.fit(X_train, y_train)

### Gradient Boosting
Gradient boosting tries to fit the new predictor to the residual errors made by the previous predictor.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X, y)

y2 = y -tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X, y2)

y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X, y3)

y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
grbt.fit(X, y)

`n_estimators`: the number of trees <br>
`learning_rate`: it sclaes the contribution of each tree. If you set it to a low value, such as 0.1, you will need more trees in the ensemble to fit the training set, but the predictions will usually generalize better. <br>
In order to find the optimal number of trees, you can use early stopping. A simple way to implement this is to use the `staged_predict()` method.

## Stacking
Each of the predictors predicts a different value, and then the final predictor (called a *blender*, or a *meta learner*) takes these predictions as inputs and makes the final prediction. <br>

Hold-out set: 
- The training set is split in two subsets. The first subset is uesded to train the predictors in the first layer.
- The first layer predictors are used to make predictions on the second (held-out) set.
- Create a new training set using predicted values as input features and keeping the target values.